In [ ]:
import pandas as pd
import csv
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

![DKB Logo](https://upload.wikimedia.org/wikipedia/commons/d/d4/Deutsche_Kreditbank_AG_Logo_2016.svg)

## Please set these parameters

In [ ]:
accountname = 'export' # set filename of the csv. e.g. 

# 0 or less means analyse all months
number_of_last_months_to_analyse = 0 # 4

### Read csv
Works for DKB csv as of 2024-10-31.

In [ ]:
csv_file= accountname+".csv"
data = pd.read_csv(csv_file, index_col="Buchungsdatum",
                   header='infer', sep=';', quoting=1,
                   parse_dates=True, date_format="%Y-%m-%d",
                   skiprows = 4, # ignore metadata which doesn't fit CSV rows format
                   keep_default_na=False, # don't make empty values to become NaN
                   encoding='utf-8', dayfirst=True)

In [ ]:
# Read metadata
data = data.iloc[::-1] # reverse rows such that bookings are chronologically sorted

with open(csv_file,"r",encoding="utf-8") as f:
    reader = csv.reader(f,delimiter=";")
    metadata = {}
    for i, row in enumerate(reader):
        if "Kontostand" in row[0]:
            value = row[1]
            value = value.replace(".","").replace(",",".")[:-2] # '12.345,67\xa0€' -> '12345.67'
            end_balance = float(value)
            break

start_date = datetime.strptime( data.index[0] ,"%d.%m.%y").date()
end_date = datetime.strptime( data.index[-1] ,"%d.%m.%y").date()

In [ ]:
print("start:",start_date)
print("end:",end_date)
print("end balance:",end_balance)

## Named Columns

In [ ]:
party = "Sender => Receiver"
sender = "Zahlungspflichtige*r"
receiver = "Zahlungsempfänger*in"
category = "Kategorie"
amount = "Betrag (€)"
cause = "Verwendungszweck"
balance = "Kontostand (EUR)"

### Cleanup

In [ ]:
# Fix US / EU decimal-point/comma
data[amount] = data[amount].str.replace('.','')
data[amount] = data[amount].str.replace(',','.')
data[amount] = data[amount].astype('float')

In [ ]:
data.head()

### Remove small transactions to avoid noise

In [ ]:
data = data[abs(data[amount]) > 0.01]

### Inverse Dataframe to have first day first and filter by time-range

In [ ]:
from dateutil.relativedelta import relativedelta

if number_of_last_months_to_analyse > 0:
    start_date = end_date - relativedelta(months=number_of_last_months_to_analyse)

# start_date may be not in the index, if there was no booking on that day. in that case, chose the next day which has any booking
if str(start_date) not in data.index:
    for dt_str in data.index:
        dt = datetime.strptime( dt_str ,"%d.%m.%y").date()
        if dt < start_date:
            continue
        start_date = dt
        print("using new start date", start_date, "which is in the index.")
        break

def dt2str(dt):
    return dt.strftime("%d.%m.%y")

print(f"Analysing time range: {start_date} -> {end_date}")
data = data.loc[dt2str(start_date):dt2str(end_date)]

start_and_end_dates_string = "(%i.%i.%i - %i.%i.%i)" % (start_date.day, start_date.month, start_date.year, end_date.day, end_date.month, end_date.year)

In [ ]:
fr = data.index[0]
to = data.index[-1]
print(fr,"->",to)

### Compute balance at each transaction

In [ ]:
data_balance = data[amount].sum().round(2)
start_balance = end_balance - data_balance
data[balance] = data[amount].cumsum()+start_balance

In [ ]:
print("start",start_balance)
print("end",end_balance)
print("balance during csv timespan",data_balance)

## Balance over time

In [ ]:
data[balance].plot(
    title='Account balance DKB %s' % accountname,
    grid=True,
    figsize=(20,8)
);

## Breakdown by transaction party

In [ ]:
data[party] = data[sender] + " => " + data[receiver]
tx_party_group = data.groupby(party).agg({amount:"sum"})

In [ ]:
tx_party_group = tx_party_group.sort_values(amount, ascending=False)

In [ ]:
shortened_party_length = 25

In [ ]:
# shorten name
tx_party_group.index = [ "=>".join(map(lambda s: s[:shortened_party_length],str(idx).split("=>"))) for idx in tx_party_group.index ]

In [ ]:
tx_party_group[amount].head()

In [ ]:
tx_party_group[tx_party_group[amount].abs() > 50].plot.barh(
    figsize=(10,60),
    title=u'Aggregierte Zahlungen ab 50€ %s' % start_and_end_dates_string
  );

In [ ]:
data.head()

## Breakdown by Category
We use some heuristics on the tranasaction details to put them into different categories.
All transactions within a category will be aggregated for a better overall analysis.
You may need to check the print output of the next cell and possibly adapt the mapping function for a better categorisation.

**the text will be lower cased before categorisation**

**Feel free to change these heuristic mappings - or adapt the code to map according to specififc transaction details.**

In [ ]:
categories = {
    "eat_out_or_get_food": [
        "restaurant",
        "gastro",
        "dean david",
        "cafe",
        "baeckerei",
        "coffee fellows",
        "jim block",
        "don qui",
        "Osteria",
        "subway",
        "backhaus",
        "burger king",
        "campus suite",
        "juice.more",
        "Backerei",
        "Avni Terhani",
        "vegan",
        "thai",
        "indisch",
    ],
    "alltag_laden": [
        "lidl",
        "aldi",
        "edeka",
        "alnatura",
        "rewe",
        "vollcorner",
    ],
    "spezial_laden": [
        "karstadt",
        "galeria",
        "kaufhof",
        "mueller",
        "migros",
        "coop",
        "dm fil",
        "go asia",
        "Drogerie",
        "SUCKFUELL",
        "butlers",
        "Suckfull",
        "Bio-Market",
        "conrad m",
    ],
    "online_handel":[
        "otto",
        "conrad elec",
        "amzn mktp",
        "amazon",
    ],
    "transport": [
        "FERNVERKEHR",
        "flixbus",
        "PAYPAL .DBVERTR",
    ],
    "ausflug": [
        "hamburg",
        "Hotel",
        "PAYPAL .booking",
        "prague",
        "praha",
        "Tubingen",
    ],
    "project": [
        "openai",
        "google.cloud",
        "github",
    ],
    "freizeit": [
        "VOLKSBAD",
        "PAYPAL .SENNHEISER",
    ],
    "entertainment": [
        "magellan",
        "Amazon Prime",
        "paypal .steam",
        "netflix"
    ],
    "crypto": [
        "bitflyer",
        "coinbase"
    ],
    "health": [
        "apotheke",
        "Krankenversicherung",
    ],
    "cash": [
        "bargeld",
        "automat",
        "cash"
    ],
    "dkb": [
        "DKB",
        "KREDITBANK",
    ],
    "miete": [],
    "investment": [],
    "emergency_fund":[],
    "uncategorized": [
        "PayPal (Europe)",
    ],
    "card_payment": [],
    "minor": [ # populated automatically
    ]
}

def mapToCategory(x):
    # use these transaction details to map to a category
    p = x[party].lower()
    c = x[cause].lower()
    
    # manual mappings
    if "WERTP. ABRECHN".lower() in c or "Depot ".lower() in c or "WERTPAPIER".lower() in c:
        return "investment"
    
    if "miete ".lower() in c:
        return "miete"
    
    if "KREDITKARTENABRECHNUNG".lower() in c:
        return "card_payment"
    
    # mappings by category
    for cat, cat_words in categories.items():
        if any(map(lambda r: r.lower() in p, cat_words)):
            return cat
    
    # debitcard. may need adaptation
    if "Debitk.20 VISA Debit".lower() in c:
        return "card_payment"
    
    return p

data[category] = data.apply(lambda x: mapToCategory(x), axis=1)

print(len(data[category].unique()),"categories")

print("============ uncategorized =================")
s = 0
for x in data[category].unique():
    ok = False
    
    for cat in categories.keys():
        if x == cat:
            ok = True

    if not ok:
        print(x)
        idx = data[category] == x
        s = s + abs(data[idx][amount].sum())
        
print("================================ sum of uncategorized: ",s) # todo. this doesn't seem to make sense ... 🤔

Breakdown by category. (Ignored transactions below 10€).

In [ ]:
byCategory = data.groupby(category).agg({amount:"sum"}).sort_values(amount,ascending=False)
byCategory = byCategory[abs(byCategory[amount]) > 10]
    

costs = byCategory[byCategory[amount] < 0]
costs.loc[:,amount] = -costs[amount]

total_costs = costs[amount].sum()
costs.plot.pie(
    figsize=(12,12),
    y=amount,
    legend=None,
    autopct=lambda x: str(round(x/100*total_costs)) + "€ | " + str(round(x)) + "%",
    title=u'Nach Kategorie Aggregierte Kosten %s' % start_and_end_dates_string
)
plt.show()

byCategory[byCategory[amount] > 0].plot.pie(
    figsize=(12,12),
    y=amount,
    legend=None,
    title=u'Nach Kategorie Aggregiertes Einkommen %s' % start_and_end_dates_string
)
plt.show()


byCategory.plot.barh(
    figsize=(6,40),
    grid=True,
    title=u'Nach Kategorie Aggregierte Zahlungen %s' % start_and_end_dates_string
)
plt.show()


### Category analysis

In [ ]:
def quick_category_analyse(cat,desc=party):
    excursions = data[data[category] == cat].sort_values(amount,ascending=True)

    excursions.plot.barh(
        figsize=(12,12),
        x=desc,
        y=amount,
        legend=None,
    )
    plt.show()

    return excursions

In [ ]:
quick_category_analyse("investment",desc=cause)

In [ ]:
quick_category_analyse("dkb", desc=cause)

In [ ]:
quick_category_analyse("ausflug")

In [ ]:
quick_category_analyse("card_payment")

In [ ]:
quick_category_analyse("transport")